In [111]:
from google.colab import drive
import zipfile
import os

# Google Drive'ı bağla
drive.mount('/content/drive')

# Dataset 1: Fresh_and_Rotten_Classification
zip_path1 = "/content/drive/MyDrive/Colab Notebooks/Fresh_and_Rotten_Classification.zip"
extract_path1 = "/content/Fresh_and_Rotten_Classification"

if not os.path.exists(extract_path1):
    with zipfile.ZipFile(zip_path1, 'r') as zip_ref:
        zip_ref.extractall(extract_path1)
    print(f"dataset1 çıkartıldı: {extract_path1}")
else:
    print(f"dataset1 zaten mevcut, tekrar çıkartılmadı: {extract_path1}")

# Dataset 2: Fruit_And_Vegetable_Diseases_Dataset
zip_path2 = "/content/drive/MyDrive/Colab Notebooks/Fruit_And_Vegetable_Diseases_Dataset.zip"
extract_path2 = "/content/Fruit_And_Vegetable_Diseases_Dataset"

if not os.path.exists(extract_path2):
    with zipfile.ZipFile(zip_path2, 'r') as zip_ref:
        zip_ref.extractall(extract_path2)
    print(f"dataset2 çıkartıldı: {extract_path2}")
else:
    print(f"dataset2 zaten mevcut, tekrar çıkartılmadı: {extract_path2}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
dataset1 zaten mevcut, tekrar çıkartılmadı: /content/Fresh_and_Rotten_Classification
dataset2 zaten mevcut, tekrar çıkartılmadı: /content/Fruit_And_Vegetable_Diseases_Dataset


In [112]:
import tensorflow as tf

def find_tf_bozuk_gorseller(root_dir):
    bozuklar = []

    for class_dir in os.listdir(root_dir):
        class_path = os.path.join(root_dir, class_dir)
        if not os.path.isdir(class_path):
            continue

        for file in os.listdir(class_path):
            file_path = os.path.join(class_path, file)
            try:
                img_raw = tf.io.read_file(file_path)
                _ = tf.image.decode_image(img_raw)  # TensorFlow'un kendi decoder'ı
            except Exception as e:
                print(f"TF bozuk: {file_path} - {e}")
                bozuklar.append(file_path)

    print(f"\nTensorFlow tarafından okunamayan dosya sayısı: {len(bozuklar)}")
    for f in bozuklar:
      try:
          os.remove(f)
          print(f"Silindi: {f}")
      except Exception as e:
          print(f"Silinemedi: {f} - {e}")

    return bozuklar

#bozuklar = find_tf_bozuk_gorseller("/content/Fruit_And_Vegetable_Diseases_Dataset/Fruit_And_Vegetable_Diseases_Dataset/Test")
#bozuklar = find_tf_bozuk_gorseller("/content/Fruit_And_Vegetable_Diseases_Dataset/Fruit_And_Vegetable_Diseases_Dataset/Train")


In [113]:
import os
import cv2
import numpy as np
from tqdm import tqdm
from skimage.feature import hog
from sklearn.decomposition import PCA
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report
from sklearn.model_selection import train_test_split
import pickle

In [114]:
def extract_hsv(img):
    hsv = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    hsv_hist = cv2.calcHist(
        [hsv], [0, 1, 2], None,
        [8, 8, 8],
        [0, 180, 0, 256, 0, 256]
    ).flatten()
    return hsv_hist

def extract_hog(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    hog_feat = hog(
        gray,
        orientations=6,
        pixels_per_cell=(8, 8),
        cells_per_block=(2, 2),
        block_norm='L2-Hys',
        feature_vector=True
    )
    return hog_feat

def extract_features(image_path):
    img = cv2.imread(image_path)
    if img is None:
        return None, None
    img = cv2.resize(img, (128, 128))
    hsv_hist = extract_hsv(img)
    hog_feat = extract_hog(img)
    return hsv_hist, hog_feat


In [115]:
def load_dataset(root_path):
    hsv_list, hog_list, labels = [], [], []
    for label in tqdm(os.listdir(root_path)):
        class_path = os.path.join(root_path, label)
        if not os.path.isdir(class_path):
            continue
        for fname in os.listdir(class_path):
            fpath = os.path.join(class_path, fname)
            hsv_hist, hog_feat = extract_features(fpath)
            if hsv_hist is not None:
                hsv_list.append(hsv_hist)
                hog_list.append(hog_feat)
                labels.append(label)
    return np.array(hsv_list), np.array(hog_list), np.array(labels)

In [116]:
def prepare_with_hog_pca_then_add_hsv(Xhsv_tr, Xhsv_val, Xhog_tr, Xhog_val, n_components=200):

    safe_nc = min(n_components, Xhog_tr.shape[1], max(1, Xhog_tr.shape[0]-1))
    pca = PCA(n_components=safe_nc)
    Xhog_tr_pca = pca.fit_transform(Xhog_tr)
    Xhog_val_pca = pca.transform(Xhog_val)

    Xtr_final  = np.concatenate([Xhog_tr_pca, Xhsv_tr], axis=1)
    Xval_final = np.concatenate([Xhog_val_pca, Xhsv_val], axis=1)
    return Xtr_final, Xval_final, pca

In [117]:
def build_final_features_with_pca(Xhsv, Xhog, fitted_pca):
    Xhog_pca = fitted_pca.transform(Xhog)
    return np.concatenate([Xhog_pca, Xhsv], axis=1)

In [118]:
def create_rf_model(n_estimators=100, max_depth=30, random_state=42):
    return RandomForestClassifier(
        n_estimators=n_estimators,
        max_depth=max_depth,
        random_state=random_state,
        class_weight="balanced_subsample",
        n_jobs=-1,
        verbose=1
    )

In [119]:
# Dataset 1: Fresh vs Rotten Dataset
print("Fresh vs Rotten Dataset")
train_dir1 = "/content/Fresh_and_Rotten_Classification/Fresh_and_Rotten_Classification/Train"

X_hsv1, X_hog1, y1 = load_dataset(train_dir1)
print("Toplam örnek:", X_hsv1.shape[0])

Xhsv_tr1, Xhsv_val1, Xhog_tr1, Xhog_val1, ytr1, yval1 = train_test_split(
    X_hsv1, X_hog1, y1, test_size=0.2, random_state=42, stratify=y1
)
print("Eğitim boyutu:", Xhsv_tr1.shape, "Doğrulama boyutu:", Xhsv_val1.shape)

Xtr1, Xval1, pca1 = prepare_with_hog_pca_then_add_hsv(
    Xhsv_tr1, Xhsv_val1, Xhog_tr1, Xhog_val1, n_components=50
)
print("X_train (birleşik):", Xtr1.shape, "X_val (birleşik):", Xval1.shape)

model1 = create_rf_model(n_estimators=100, max_depth=30)
model1.fit(Xtr1, ytr1)
y_pred1 = model1.predict(Xval1)

with open("rf_fresh_and_rotten.pkl", "wb") as f:
    pickle.dump(model1, f)

with open("pca_fresh_and_rotten.pkl", "wb") as f:
    pickle.dump(pca1, f)

print("\nDataset 1: Fresh vs Rotten (Validation)")
print("Doğruluk:", accuracy_score(yval1, y_pred1))
print(classification_report(yval1, y_pred1))

Fresh vs Rotten Dataset


100%|██████████| 15/15 [02:42<00:00, 10.84s/it]


Toplam örnek: 21044
Eğitim boyutu: (16835, 512) Doğrulama boyutu: (4209, 512)
X_train (birleşik): (16835, 562) X_val (birleşik): (4209, 562)


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    0.8s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    2.0s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.0s finished



Dataset 1: Fresh vs Rotten (Validation)
Doğruluk: 0.9828937990021382
                precision    recall  f1-score   support

   freshapples       1.00      0.99      1.00       485
   freshbanana       1.00      1.00      1.00       494
 freshcucumber       0.94      0.88      0.91        99
     freshokra       0.86      0.95      0.90       127
  freshoranges       1.00      1.00      1.00       293
   freshpotato       0.85      0.89      0.87       107
   freshtomato       1.00      1.00      1.00       372
  rottenapples       1.00      1.00      1.00       650
  rottenbanana       0.99      1.00      1.00       586
rottencucumber       0.97      0.82      0.89        84
    rottenokra       0.94      0.97      0.96        68
 rottenoranges       1.00      1.00      1.00       319
  rottenpotato       0.90      0.88      0.89       160
  rottentomato       1.00      0.99      1.00       365

      accuracy                           0.98      4209
     macro avg       0.96      0

In [120]:
# Dataset 2: Fruit and Vegetable Diseases Dataset
print("\nFruit and Vegetable Diseases Dataset")
train_dir2 = "/content/Fruit_And_Vegetable_Diseases_Dataset/Fruit_And_Vegetable_Diseases_Dataset/Train"

X_hsv2, X_hog2, y2 = load_dataset(train_dir2)
print("Toplam örnek:", X_hsv2.shape[0])

Xhsv_tr2, Xhsv_val2, Xhog_tr2, Xhog_val2, ytr2, yval2 = train_test_split(
    X_hsv2, X_hog2, y2, test_size=0.2, random_state=42, stratify=y2
)
print("Eğitim boyutu:", Xhsv_tr2.shape, "Doğrulama boyutu:", Xhsv_val2.shape)

Xtr2, Xval2, pca2 = prepare_with_hog_pca_then_add_hsv(
    Xhsv_tr2, Xhsv_val2, Xhog_tr2, Xhog_val2, n_components=50
)
print("X_train (birleşik):", Xtr2.shape, "X_val (birleşik):", Xval2.shape)

model2 = create_rf_model(n_estimators=100, max_depth=30)
model2.fit(Xtr2, ytr2)
y_pred2 = model2.predict(Xval2)

with open("rf_fruit_and_vegetable.pkl", "wb") as f:
    pickle.dump(model2, f)

with open("pca_fruit_and_vegetable.pkl", "wb") as f:
    pickle.dump(pca2, f)

print("\nDataset 2: Fruit & Vegetable Diseases (Validation)")
print("Doğruluk:", accuracy_score(yval2, y_pred2))
print(classification_report(yval2, y_pred2))


Fruit and Vegetable Diseases Dataset


100%|██████████| 29/29 [03:21<00:00,  6.96s/it]


Toplam örnek: 23425
Eğitim boyutu: (18740, 512) Doğrulama boyutu: (4685, 512)
X_train (birleşik): (18740, 562) X_val (birleşik): (4685, 562)


[Parallel(n_jobs=-1)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done  34 tasks      | elapsed:    1.1s
[Parallel(n_jobs=-1)]: Done 100 out of 100 | elapsed:    2.9s finished
[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.0s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.0s finished



Dataset 2: Fruit & Vegetable Diseases (Validation)
Doğruluk: 0.9216648879402348
                      precision    recall  f1-score   support

      Apple__Healthy       0.96      0.98      0.97       390
       Apple__Rotten       0.93      0.96      0.94       468
     Banana__Healthy       0.97      0.95      0.96       320
      Banana__Rotten       0.89      0.99      0.94       448
 Bellpepper__Healthy       0.82      0.88      0.85        98
  Bellpepper__Rotten       0.60      0.60      0.60        95
     Carrot__Healthy       0.91      0.86      0.89        99
      Carrot__Rotten       0.79      0.59      0.67        93
   Cucumber__Healthy       0.80      0.91      0.85        97
    Cucumber__Rotten       0.95      0.88      0.92        95
      Grape__Healthy       0.94      0.91      0.92        32
       Grape__Rotten       0.91      0.91      0.91        32
      Guava__Healthy       1.00      1.00      1.00        32
       Guava__Rotten       0.96      0.72      0.8

In [121]:
# Dataset 1: Fresh vs Rotten Dataset
test_dir1 = "/content/Fresh_and_Rotten_Classification/Fresh_and_Rotten_Classification/Test"

X_hsv1_te, X_hog1_te, y1_te = load_dataset(test_dir1)
X1_te = build_final_features_with_pca(X_hsv1_te, X_hog1_te, pca1)

y1_te_pred = model1.predict(X1_te)

print("\n[FINAL TEST] Dataset 1: Fresh vs Rotten (Test)")
print("Test doğruluk:", accuracy_score(y1_te, y1_te_pred))
print(classification_report(y1_te, y1_te_pred))

100%|██████████| 15/15 [00:48<00:00,  3.23s/it]



[FINAL TEST] Dataset 1: Fresh vs Rotten (Test)
Test doğruluk: 0.9928762243989314
                precision    recall  f1-score   support

   freshapples       1.00      1.00      1.00       791
   freshbanana       1.00      1.00      1.00       892
 freshcucumber       0.99      0.98      0.98       279
     freshokra       0.97      0.99      0.98       370
  freshoranges       0.99      1.00      1.00       388
   freshpotato       0.97      0.97      0.97       270
   freshtomato       1.00      1.00      1.00       255
  rottenapples       1.00      1.00      1.00       988
  rottenbanana       1.00      1.00      1.00       900
rottencucumber       1.00      0.95      0.97       255
    rottenokra       1.00      1.00      1.00       224
 rottenoranges       1.00      0.99      1.00       403
  rottenpotato       0.97      0.98      0.97       370
  rottentomato       1.00      1.00      1.00       353

      accuracy                           0.99      6738
     macro avg      

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.1s finished


In [122]:
# Dataset 2: Fruit and Vegetable Diseases Dataset
test_dir2 = "/content/Fruit_And_Vegetable_Diseases_Dataset/Fruit_And_Vegetable_Diseases_Dataset/Test"

X_hsv2_te, X_hog2_te, y2_te = load_dataset(test_dir2)
X2_te = build_final_features_with_pca(X_hsv2_te, X_hog2_te, pca2)

y2_te_pred = model2.predict(X2_te)

print("\n[FINAL TEST] Dataset 2: Fruit & Vegetable Diseases (Test)")
print("Test doğruluk:", accuracy_score(y2_te, y2_te_pred))
print(classification_report(y2_te, y2_te_pred))

100%|██████████| 29/29 [00:53<00:00,  1.86s/it]



[FINAL TEST] Dataset 2: Fruit & Vegetable Diseases (Test)
Test doğruluk: 0.9266541289109249
                      precision    recall  f1-score   support

      Apple__Healthy       0.96      0.98      0.97       487
       Apple__Rotten       0.92      0.96      0.94       585
     Banana__Healthy       0.94      0.97      0.96       399
      Banana__Rotten       0.92      0.99      0.95       559
 Bellpepper__Healthy       0.83      0.93      0.88       122
  Bellpepper__Rotten       0.62      0.62      0.62       117
     Carrot__Healthy       0.92      0.85      0.89       124
      Carrot__Rotten       0.78      0.64      0.70       116
   Cucumber__Healthy       0.80      0.89      0.84       121
    Cucumber__Rotten       0.90      0.76      0.83       118
      Grape__Healthy       0.92      0.90      0.91        40
       Grape__Rotten       0.88      0.93      0.90        40
      Guava__Healthy       0.95      1.00      0.98        40
       Guava__Rotten       0.94      0

[Parallel(n_jobs=8)]: Using backend ThreadingBackend with 8 concurrent workers.
[Parallel(n_jobs=8)]: Done  34 tasks      | elapsed:    0.1s
[Parallel(n_jobs=8)]: Done 100 out of 100 | elapsed:    0.1s finished
